In [ ]:
import agentpy as ap
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import IPython
import math
import random

In [ ]:
def normalize(v):
    """ Normalize a vector to length 2. """
    norm = np.linalg.norm(v)
    if norm == 0:
        return v
    return math.sqrt(2) * v / norm

In [ ]:
p = {
    'size': 201,
    'seed': 123,
    'steps': 80,
    'population': 1000,
    'nevent': 250,
    'pertlucky': 0.5,
    'talentmean': 0.6,
    'talentsd': 0.1,
    'radius': 2,
    'intialcap': 10,
}

In [ ]:
class individual(ap.Agent):
    """ An agent with a position and velocity in a continuous space,
    who follows Craig Reynolds three rules of flocking behavior;
    plus a fourth rule to avoid the edges of the simulation space. """
    
    def setup_talent(self):
        self.talent = model.nprandom.normal(self.p.talentmean, self.p.talentsd)
        
    def setup_capital(self):
        self.capital = self.p.intialcap
    
    def setup_pos(self, space):
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def take_action(self):
        intersects = self.neighbors(self, distance=self.p.radius)
        for nb in intersects:
            if nb.type == "lucky_event":
                if self.talent > random.uniform(0, 1):
                    self.capital *= 2
#                     print("lucky: ", self.capital)
            if nb.type == "unlucky_event":
                self.capital /= 2
#                 print("unlucky: ", self.capital)
        
        
class lucky_event(ap.Agent):
    
    def setup(self):
        self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
    
    def update_velocity(self):
        self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
    
    def setup_type(self):
        self.velocity
    
    def setup_pos(self, space):       
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
        
    def update_position(self):
        self.space.move_by(self, self.velocity)
        
    def take_action(self):
        pass
        

class unlucky_event(ap.Agent):
    
    def setup(self):
        self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
    
    def update_velocity(self):
        self.velocity = normalize(self.model.nprandom.random(2) - 0.5)
    
    def setup_pos(self, space):       
        self.space = space
        self.neighbors = space.neighbors
        self.pos = space.positions[self]
    
    def update_position(self):
        self.space.move_by(self, self.velocity)
        
    def take_action(self):
        pass

In [ ]:
class TvLModel(ap.Model):
    
    def setup(self):

        self.space = ap.Space(self, shape=[self.p.size]*2)

        self.people = ap.AgentList(self, self.p.population, individual)
        self.space.add_agents(self.people, random=True)
        self.people.setup_talent()
        self.people.setup_capital()
        self.people.setup_pos(self.space)
        
        self.lucky_events = ap.AgentList(self, self.p.nevent, lucky_event)
        self.space.add_agents(self.lucky_events, random=True)
        self.lucky_events.setup_pos(self.space)

        self.unlucky_events = ap.AgentList(self, self.p.nevent, unlucky_event)
        self.space.add_agents(self.unlucky_events, random=True)
        self.unlucky_events.setup_pos(self.space)
        
    def step(self):
        self.lucky_events.update_velocity()
        self.unlucky_events.update_velocity()
        self.lucky_events.update_position()
        self.unlucky_events.update_position()
        self.people.take_action()

    def update(self):
        self.record('individual capital', self.people.capital)

    def end(self):
        self.people.record('capital')
        self.people.record('talent')


In [ ]:
def animation_plot_single(m, ax):

    ax.set_title(f"Boids Flocking Model t={m.t}")
    ppl_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'individual']
    ppl_pos = np.array(list(ppl_pos)).T  # Transform
    
    lucky_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'lucky_event']
    lucky_pos = np.array(list(lucky_pos)).T  # Transform
    
    unlucky_pos = [pos for agent, pos in m.space.positions.items() if agent.type == 'unlucky_event']
    unlucky_pos = np.array(list(unlucky_pos)).T  # Transform
    
    ax.scatter(*ppl_pos, s=200, c='black', marker="x")
    ax.scatter(*lucky_pos, s=200, c='green', marker="o")
    ax.scatter(*unlucky_pos, s=200, c='red', marker="o")
    ax.set_xlim(0, 50)
    ax.set_ylim(0, 50)
    ax.set_axis_off()

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.add_subplot(111, projection=None)
animation = ap.animate(TvLModel(p), fig, ax, animation_plot_single, steps=80)
IPython.display.HTML(animation.to_jshtml(fps=15))

In [ ]:
model = TvLModel(p)
results = model.run()

In [ ]:
data = results.variables.individual
data = data.reset_index()[['obj_id', 'capital', 'talent']]

In [ ]:
data = data.set_index('obj_id')

In [ ]:
data.sort_values('talent', ascending=False)

In [ ]:
data.sort_values('capital', ascending=False)